In [1]:
### Commencer par défénir les bibliothèques utilisées
import numpy as np

In [2]:
### L'architecture de RNA choisie pour le problème de XOR
## Nous aurons comme entrée 2 neurones et comme sortie : soit 1 neurone soit 2 neurones de sortie
## On définit alors les paramètres de réseaux de neurones comme suit:
RNA=[]
W1=np.array([[0.7,-0.2],[-0.4,0.3]])
b1=np.array([1,0])
W2=np.array([[0.5,0.1]])
b2=np.array([1])
RNA.append([W1,b1])
RNA.append([W2,b2])
#### 
print(RNA[0][0])

[[ 0.7 -0.2]
 [-0.4  0.3]]


In [3]:
#### La base de données
X=np.array([[1,0],[0,0],[0,1],[1,1]])
Y=np.array([1,0,1,0])

In [4]:
## La fonction neurone
def neurone(x,g,w,b):
    z=np.dot(x,w)+b
    return np.array([z,g(z)])

def activ(s):
    return 1/(1+np.exp(-s))

In [5]:
print(neurone(X[0],activ,RNA[0][0][0],RNA[0][1][0]))### la sortie de neurone 1 de la couche caché
print(neurone(X[0],activ,RNA[0][0][1],RNA[0][1][1]))### la sortie de neurone 2 de la couche caché

[1.7        0.84553473]
[-0.4         0.40131234]


In [6]:
#### La fonction Couche
def couche(x,g,W,b):
    z=[]
    a=[]
    for i in range(W.shape[0]):
        z.append(neurone(x,g,W[i],b[i])[0])
        a.append(neurone(x,g,W[i],b[i])[1])
    return np.array(z),np.array(a)

In [7]:
z1,a1=couche(X[0],activ,RNA[0][0],RNA[0][1])
z2,a2=couche(a1,activ,RNA[1][0],RNA[1][1])
print(a1)
print(z1)
print(a2)
print(z2)

[0.84553473 0.40131234]
[ 1.7 -0.4]
[0.81197561]
[1.4628986]


In [110]:
### Définir l'algorithme de propagation avant pour un RNA pour un exemple x
def prop_avant(x,RNA):
    a=[]*(len(RNA)+1)
    z=[]*(len(RNA)+1)
    a.append(np.array(x))
    z.append(np.array([x]))
    
    for i in range(len(RNA)):
        #W=RNA[i][0]
        #b=RNA[i][1]
        #couche(a[i][1],activ,W,b)[0]
        
        z1,a1=couche(a[i],activ,RNA[i][0],RNA[i][1])
        z.append(z1)
        a.append(a1)
        #print("la parition lineaire {} de couche {}".format(z[i],i))
        #a.append(couche(a[i][1],activ,W,b)[1])
        
    return z,a

In [111]:
z,a=prop_avant(X[0],RNA)
print(a)
print(z)
print(z[0].T)

[array([1, 0]), array([0.15446527, 0.59868766]), array([0.24285295])]
[array([[1, 0]]), array([ 1.7, -0.4]), array([1.1371014])]
[[1]
 [0]]


In [68]:
### ### fonction cout
def cout_classification(a,y):
    return np.sum(-y*np.log(a)-(1-y)*np.log(1-a))
###
### Définir une fonction de moyenne d'erreurs

def erreur_moy(RNA,cout,X,Y):
    err=0
    for i in range(len(Y)): 
        ### Y.shape[0]
        a=prop_avant(X[i],RNA)
        err+=cout(a[1][-1],Y[i])*(1/len(Y))
    return err

In [73]:
z,a=prop_avant(X[0],RNA)
print(cout_classification(a[-1],Y[0]))
z,a=prop_avant(X[1],RNA)
print(cout_classification(a[-1],Y[1]))
z,a=prop_avant(X[2],RNA)
print(cout_classification(a[-1],Y[2]))
z,a=prop_avant(X[3],RNA)
print(cout_classification(a[-1],Y[3]))
print(erreur_moy(RNA,cout_classification,X,Y))

1.4152991828985817
0.2668986229538734
1.4614143313854782
0.2765967122494966
0.8550522123718575


In [78]:
#### la retro_propagation des gradients
### On commence de definir la fonction dérivée de l'activation
def dev_g(s):
    return np.exp(-s)/(1-np.exp(-s))**2

##########################################
def retro_prop_c_sortie(RNA,a,z,y):
    #da2=-y/a2+(1-y)/(1-a2)
    da2=-y/a[-1]+(1-y)/(1-a[-1])
    #print(da2)
    #dz2=da2*dev_g(z2)
    dz2=da2*dev_g(z[-1])
    #print(dz2)
    #dw2=dz2*a1
    dW2=dz2*a[1]
    #print(dW2)
    db2=dz2
    #da1=W2*dz2
    W=RNA[1][0][0]
    da1=W*dz2
    return da1,dW2,db2

In [79]:
z,a=prop_avant(X[0],RNA)
[da1,dW2,db2]=retro_prop_c_sortie(RNA,a,z,Y[0])
print(da1)
print(dW2)
print(db2)

[-1.43128837 -0.28625767]
[-0.44216867 -1.71378936]
[-2.86257673]


In [112]:
def retro_prop_c_cache(RNA,a,z,da1):
    #dz2=da2*dev_g(z2)
    dz1=da1*dev_g(z[1])
    print(dz1)
    #dw2=dz2*a1
    #dW1=dz1*a[1].T
    dW1=dz1*z[0].T
    #print(dW1)
    db1=dz1
    
    return dW1,db1

In [113]:
retro_prop_c_cache(RNA,a,z,da1)
#dW1,db1=retro_prop_c_cache(RNA,a,da1)

[-0.39142271 -1.76544528]


(array([[-0.39142271, -1.76544528],
        [-0.        , -0.        ]]),
 array([-0.39142271, -1.76544528]))

In [ ]:
##### L'apprentissage de réseau
#### la mise à jour de W et b
#### alpha, n iter maxi, X,y,errre, prop avant, retropro,

def Apprentissage_RNA(RNA,alpha,n_it,X,y):
    err=[]
    for _ in range(n_it):
        for i in range (X.shape[0])
        a=prop_avant(X[i],RNA)
        err+=cout(a[1][-1],Y[i])*(1/len(Y))
        